# enriching Mind

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# New call
from pipeline_newcall import generate_groups
import sys
import pickle, json
import numpy as np
from scipy.spatial.distance import cosine
from collections import Counter
sys.path.append("/home/ray__/ssd/ai-engine/pkg/")
import text_preprocessing.preprocess as tp
from extra_preprocess import preprocess_text

sys.path.append("helper_functions/")
from generate_controlled_top_10_ent import get_ent

sys.path.append("/home/ray__/ssd/BERT/")
from gpt_feat_utils import GPT_Inference
gpt_model = GPT_Inference("/home/ray__/ssd/BERT/models/se/epoch3/", device="cpu")

# S.E
ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/se/se_entity_feats_v3.pkl","rb"))
sent_dict = pickle.load(open("/home/ray__/ssd/minds/se/se_sent_dict_v3.pkl", "rb"))
com_map = pickle.load(open("/home/ray__/ssd/minds/se/com_map.pkl", "rb"))
kp_entity_graph = pickle.load(open("/home/ray__/ssd/minds/se/deployment/kp_entity_graph.pkl", "rb"))


common_entities = ent_fv_full.keys() & com_map.keys()
ent_fv = {}
for ent in common_entities:
    ent_fv[ent] = ent_fv_full[ent]
    


In [ ]:
from copy import deepcopy
def get_ranked_groups(request, ent_fv, com_map, kp_entity_graph):
    group, group_ent = get_ent(request, ent_fv, com_map, kp_entity_graph)
    return group_ent

def get_entity_mapping(group_ent, ent_fv, com_map, kp_entity_graph):
    group_ent_map = {}
    for groupid, ent_list in group_ent.items():
        group_ent_map[groupid] = [com_map[ent] for ent in list(map(lambda kv:kv[0], group_ent[groupid]))]
        #group_ent_score[groupid] = [ranked_com[com] for com in group_ent_score[groupid] if com in ranked_com.keys()]
    
    return group_ent_map

def filter_entity_community(group_ent_map, ent_fv, com_map, kp_entity_graph):
    group_ent_map_filtered_intrm = {}
    group_ent_map_filtered = {}
    for groupid, ent_map in group_ent_map.items():
        filtered_ent_map = []
        if len(set(ent_map)) == len(ent_map):
            group_ent_map_filtered[groupid] = []
        else:
            count_a = Counter(ent_map).most_common()
            for i, count in count_a:
                if count>1 :
                    filtered_ent_map.append((i, count))

            group_ent_map_filtered[groupid] = filtered_ent_map

    return group_ent_map_filtered


def get_entity_mapping_rank(group_ent_map_filtered, gc, lc, ent_fv, com_map, kp_entity_graph):
    group_ent_map_rank_lc = {}
    group_ent_map_rank_gc = {}
    for groupid, ent_map_list in group_ent_map_filtered.items():
        group_ent_map_rank_intrm_lc = []
        group_ent_map_rank_intrm_gc = []
        
        for ent_map, count in ent_map_list:
            if ent_map in lc.keys() and sum(lc[ent_map])!=0:
                group_ent_map_rank_intrm_lc.append(sum(lc[ent_map]))
            else:
                if ent_map in gc.keys():
                    group_ent_map_rank_intrm_gc.append(gc[ent_map])
                else:
                    group_ent_map_rank_intrm_gc.append(0)
        if group_ent_map_rank_intrm_lc!=[]:
            group_ent_map_rank_lc[groupid] = sum(group_ent_map_rank_intrm_lc)
        else:
            group_ent_map_rank_gc[groupid] = sum(group_ent_map_rank_intrm_gc) 
            
            
    ## update gc and lc
    updated_lc_list = []
    for groupid, ent_map_list in group_ent_map_filtered.items():
        for ent_map, count in ent_map_list:
            if ent_map in lc.keys():
                if len(lc[ent_map])!=5:
                    lc[ent_map].append(count)
                else:
                    del lc[ent_map][0]
                    lc[ent_map].append(count)
                updated_lc_list.append(ent_map)
            else:
                lc[ent_map] = [count]
                updated_lc_list.append(ent_map)
                
            if ent_map in gc.keys():
                gc[ent_map] +=count
            else:
                gc[ent_map] = count
                
    print (updated_lc_list)
    lc_copy = deepcopy(list(lc.items()))            
    for ent, freq in lc_copy:
        if ent not in updated_lc_list:
            if sum(lc[ent]) == 0:
                del lc[ent]
            else:
                if len(lc[ent])!=5:
                    lc[ent].append(0)
                else:
                    del lc[ent][0]
                    lc[ent].append(0)
    
    return group_ent_map_rank_lc, group_ent_map_rank_gc, gc, lc 


def compute_groups_new_call(req, ent_fv, com_map, kp_entity_graph, gc, lc, store=False):
    group, group_ent = get_ranked_groups(req, ent_fv, com_map, kp_entity_graph)
    group_ent_map = get_entity_mapping(group_ent, ent_fv, com_map, kp_entity_graph)
    group_ent_map_filtered = filter_entity_community(group_ent_map, ent_fv, com_map, kp_entity_graph)
    group_ent_map_rank_lc, group_ent_map_rank_gc, gc_copy, lc_copy = get_entity_mapping_rank(group_ent_map_filtered, gc, lc, ent_fv, com_map, kp_entity_graph)
    if store:
        print ("writing the gc and lc update.")
        pickle.dumps(gc, open("/home/ray__/ssd/minds/se/recency/gc.pkl","wb"))
        pickle.dumps(lc, open("/home/ray__/ssd/minds/se/recency/lc.pkl","wb"))
    return group, group_ent_map_rank_lc, group_ent_map_rank_gc, gc_copy ,lc_copy

In [ ]:
test_set = ['topic_testing/sync_eng_2020_01_10.txt']
for file in test_set:
    gc = pickle.load(open("/home/ray__/ssd/minds/se/recency/gc.pkl","rb"))
    lc = pickle.load(open("/home/ray__/ssd/minds/se/recency/lc.pkl","rb"))
    with open(file,'rb') as f:
        request = json.load(f)
        if isinstance(request, str):
            request = json.loads(request)

    #ranked_groups, group_map, group_rank, group_ent, gc, lc = compute_groups_new_call(request)
    group, group_ent_map_rank_lc, group_ent_map_rank_gc, gc, lc = compute_groups_new_call(request, ent_fv, com_map, kp_entity_graph, gc, lc)

In [ ]:
gc

In [ ]:
lc

In [ ]:
lc